<a href="https://colab.research.google.com/github/hyesungKomet/rokaf_ai/blob/main/Elice_3_4_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO(You Only Look Once)

* 2-stage detector:  
region proposal(객체로 의심되는 후보영역 찾기)을 한 뒤에 classification(후보별로 이미지 분류)이 이루어짐  
 -> Accuracy는 높지만 FPS(속도)가 느림
ex) Faster R-CNN이 5 FPS인데 Real-time detection을 위해선 적어도 30 FPS가 필요하다고 함

* 1-stage detector:  
region proposal과 classification을 한번에 수행
기존의 region proposal 방식이 아닌 grid 도입으로 속도 향상


![](https://www.haxed.dk/tablesoccer/public/img/yolo-architecture.png)

* 7 x 7 x 30 
bounding box 2개의 좌표와 confidence값, 각 class의 확률 값
![](https://user-images.githubusercontent.com/15168540/48966993-9a679e80-f01d-11e8-8f78-66a7135859eb.png)

  * x: 각 grid cell의 TopLeft부터의 x좌표
  * y: 각 grid cell의 TopLeft부터의 y좌표
  * w: 전체 image의 width와 박스의 width의 비율
  * h: 전체 image의 height와 박스의 height의 비율  
  x,y가 bounding box의 가운데 좌표를 의미, w,h가 bounding box의 가로, 세로 길이를 의미
  * C: Confidence, objectness - 해당 grid cell에 객체 존재할 지의 확률 값
  * Class Probabilities: 해당 grid cell에서 각 class(자전거, 버스, 사람 등)일 확률 값

Bounding Box에 해당하는 값 5개가 Anchor Box가 된다



## Loss
정답 box는 7 x 7 x 25
정답인 box가 하나이므로!  

* box objectness + box location + class confidence

## Grid

![](https://mblogthumb-phinf.pstatic.net/MjAxNzA0MjhfMjUw/MDAxNDkzMzY0NDQyNDQ5.TF2xVDnVt0LLsAqLFRmBLAxsPa1W396-hPRwiHksaoUg.TF8F434-MLBQjVvU02dXT1fduB23VquAny80Kiex4KEg.PNG.sogangori/interpret0.PNG?type=w2)

두 anchor box의 PC(predicted confidence)를 각 class probabilites에 곱해서 20x1 행렬 두 개를 만든다  
objectness와 class confidence 모두 높아야 성능이 좋기 때문!!  
  
1. grid가  7 x 7 이니 총 7 x 7 x 2 = 98 개의 20 x 1 행렬이 나온다

  너무 많음!!

  점수에 따른 바운딩 박스 필터링 후 NMS 적용!

2. 20개의 class 모두에서 Class Confidence가 threshold보다 낮으면 0으로 변경
3. Class Confidence를 내림차순으로 정렬(Class별로)
4. 맨 앞에 가장 Confidence 높은 box가 온다
5. 첫 Class Confidence와 다른 박스들의 IoU를 구해서 IoU threshold(보통 0.5)이상이면 0으로 바꿔준다 - Class별로니까 20번 수행하겠지?

6. 이렇게 하면 98개 중 몇 개만 남을텐데, 각 박스들에서 가장 큰 Class Confidence에 해당하는 값을 지정하고 그 값(Score)이 0보다 크면 해당 Bounding Box는 그 Class를 인식하는 박스로 그리게 된다

![](https://i.ibb.co/JKgny4t/image.png)

## Yolo Layer 만들기

In [ ]:
from tensorflow.keras import datasets, layers, models, activations, losses, optimizers, metrics
from keras.layers import LeakyReLU

def create_yolo():
    model = models.Sequential()
    
    # Block1
    model.add(layers.Convolution2D(64, (7, 7), strides=(2, 2), input_shape=(448, 448, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    # Block2
    model.add(layers.Convolution2D(192, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))    
    
    # Block3
    model.add(layers.Convolution2D(128, (1, 1), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(256, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(256, (1, 1), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(512, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))    
    
    # Block4
    model.add(layers.Convolution2D(256, (1, 1), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(512, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(256, (1, 1), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(512, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(256, (1, 1), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(512, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(256, (1, 1), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(512, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(512, (1, 1), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(1024, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    # Block5
    model.add(layers.Convolution2D(512, (1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(1024, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(512,(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(1024, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(1024,(3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(1024,(3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.1))

    # Block6
    model.add(layers.Convolution2D(1024,(3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.Convolution2D(1024, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))

    # Last Block
    model.add(layers.Flatten())
    model.add(layers.Dense(4096))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(7 * 7 * 30))
    model.add(layers.Reshape(target_shape=(7, 7, 30)))
    
    return model


In [ ]:
yolo= create_yolo()
yolo.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 224, 224, 64)      9472      
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 224, 224, 64)      0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 conv2d_18 (Conv2D)          (None, 112, 112, 192)     110784    
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 112, 112, 192)     0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 56, 56, 192)      0         
 2D)                                                  

## Pascal VOC Dataset load
dataset.py
xml 파일 다루기

In [ ]:
import os
import glob
import cv2
import numpy
import xml.etree.ElementTree as ET
import tqdm

classes_num = {'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4, 
               'bus': 5, 'car': 6, 'cat': 7, 'chair': 8, 'cow': 9, 
               'diningtable': 10, 'dog': 11, 'horse': 12, 'motorbike': 13, 'person': 14, 
               'pottedplant': 15, 'sheep': 16, 'sofa': 17, 'train': 18, 'tvmonitor': 19}

classes = list(classes_num.keys())


def voc_load_data(img_dir_path, annotation_path, batch=10):
    images, labels = [], []
    img_file_list = glob.glob((img_dir_path + "/*.jpg"))

    for i in range(len(img_file_list)):
        for img_path in tqdm.tqdm(img_file_list[batch * i: batch * (i + 1)]):

            # Read image
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_h, image_w = image.shape[0:2]

            # Resize (x, y)크기 이미지 -> (488, 488)로 변경
            image = cv2.resize(image, (488,488))
            # normalization 이미지 데이터를 0~1 사이의 값으로 변경 - 정규화 진행
            image = image / 255.0

            images.append(image)

            # Read xml file
            xml_name = os.path.split(img_path)[-1]
            xml_name = xml_name.split(".")[-2]
            xml_path = annotation_path + f"/{xml_name}.xml"

            # parse xml
            tree = ET.parse(xml_path)
            root = tree.getroot()

            # Empty matrix, (7, 7, 25)크기의 0으로 채워진 label_matrix 
            label_matrix = numpy.zeros((7, 7, 25)) # 정답에 해당하는 레이블(은 바운딩 박스가 하나다)

            for obj in root.iter('object'):
                difficult = obj.find('difficult').text
                class_name = obj.find('name').text
                if class_name not in classes or difficult == "1":
                    continue

                # Set class id
                cls_id = classes.index(class_name)
                xmlbox = obj.find('bndbox')
                tlx, tly = int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text)
                brx, bry = int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text)

                # x좌표, y좌표를 0~1 사이의 값으로 일반화(정규화)
                x = (brx + tlx) / 2 / image_w #가운데 좌표(스케일)
                y = (bry + tly) / 2 / image_h
                
                # w, h를 0~1 사이의 값으로 일반화(정규화)
                w = (brx - tlx) / image_w # 가로 길이
                h = (bry - tly) / image_h # 세로 길이

                # (7x7)그리드 셀의 좌표와 셀 안에서의 좌표
                loc = [7 * x, 7 * y]
                loc_i = int(loc[1]) # 그리드 좌표 y 7x7 그리드기에 실수에서 정수로 딱 바꿔줘야 함
                loc_j = int(loc[0]) # 그리드 좌표 x
                y = loc[1] - loc_i # 그리드 셀 안에서의 y좌표
                x = loc[0] - loc_j # 그리드 셀 안에서의 x좌표

                if label_matrix[loc_i, loc_j, 24] == 0:
                    # [<----------20---------->|x|y|w|h|pc]
                    label_matrix[loc_i, loc_j, cls_id] = 1 #정답데이터니까 객체 class도 바르게 맞췄겠지
                    # 20 class 중 해당하는 하나에 1 넣어주는것
                    label_matrix[loc_i, loc_j, 20:24] = [x, y, w, h]
                    label_matrix[loc_i, loc_j, 24] = 1  # response - 정답데이터이기에 PC값이 무조건 1이지

            labels.append(label_matrix)

        return numpy.array(images), numpy.array(labels)

In [ ]:
import tensorflow
from tensorflow.keras import datasets, layers, models, activations, losses, optimizers, metrics, utils
import model
import loss
import dataset

if __name__ == "__main__":

    train_images, train_labels = dataset.voc_load_data("./VOC2007/images", "./VOC2007/labels")    

    yolo = model.create_yolo()

    with tensorflow.device("/cpu:0"):
        yolo.compile(optimizer=optimizers.Adam(), loss=loss.yolo_loss)
        yolo.fit(train_images, train_labels, epochs=1, verbose=2)
        result = yolo.evaluate(train_images, train_labels)
        print(result)

    # 수정하지 마세요. 채점에 사용되는 코드입니다.
    print(train_images[0, :, :, :].sum())
    print(train_labels[0, :, :, :].sum())


## Generator 사용해서 Batch 단위로 학습하기
data_generator.py

In [ ]:
from tensorflow import keras
import math
import cv2 as cv
import numpy as np
import numpy
import xml.etree.ElementTree as ET
import glob
import os


classes_num = {'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4, 'bus': 5,
               'car': 6, 'cat': 7, 'chair': 8, 'cow': 9, 'diningtable': 10, 'dog': 11,
               'horse': 12, 'motorbike': 13, 'person': 14, 'pottedplant': 15, 'sheep': 16,
               'sofa': 17, 'train': 18, 'tvmonitor': 19}

classes = list(classes_num.keys())


class SequenceData(keras.utils.Sequence): # 데이터를 연속적으로 batch로 잘라서 처리하도록 한다

    def __init__(self, model, img_dir_path, annotation_path, target_size, batch_size, shuffle=True):
        self.model = model
        self.datasets = glob.glob((img_dir_path + "/*.jpg")) #여기 조정해서 train, test, val dataset 다르게 설정 가능
        self.image_path = img_dir_path
        self.label_path = annotation_path
        self.image_size = target_size[0:2]
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.datasets)) # __getitem__에 쓰임(아마도)
        self.shuffle = shuffle

    def __len__(self): #배치 당 데이터셋의 길이
        num_imgs = len(self.datasets)
        return math.ceil(num_imgs / float(self.batch_size)) # 1000개를 100개씩 10번 묶으면 100 반환

    def __getitem__(self, idx): # iterable한 객치(튜플, 리스트 등등)에서 아이템 가져다주는 함수
        batch_index = self.indexes[idx * self.batch_size: ((idx + 1) * self.batch_size)] # 101~200까지 슬라이싱
        batch = [self.datasets[k] for k in batch_index] # 101~200번째의 파일경로가 들어있음
        # idx는 idx 번째 배치 데이터 셋을 의미합니다.
        # idx = 2라면, 전체 데이터 셋 / 배치 사이즈 하였을 때 2번째 묶음을 의미
        # ex) 1000개 100개씩 묶었음 101~200까지의 데이터들
        
        # 아래 함수의 인자로 들어가는 batch는 전체 데이터셋 이미지 경로들 중 일부가 담rla
        # 알맞은 갯수의 이미지와 대응하는 라벨값을 전달
        X, y = self.data_generation(batch)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes) #인덱스를 섞어서 getitem에서 바꿔서 가져오게 한다(shuffle의 내부적인 부분)

    def read(self, image_path): #이건 한 데이터에 대한 처리(dataset.py는 전체를 for문으로 돌았음)
        xml_path = os.path.join(os.path.abspath(self.label_path),
                                os.path.split(image_path)[-1].split('.')[0]) + ".xml"

        image = cv.imread(image_path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image_h, image_w = image.shape[0:2]
        image = cv.resize(image, self.image_size)
        image = image / 255.

        tree = ET.parse(xml_path)
        root = tree.getroot()
        label_matrix = numpy.zeros([7, 7, 25])

        label_matrix = np.zeros([7, 7, 25])
        for obj in root.iter('object'):
            difficult = obj.find('difficult').text
            class_name = obj.find('name').text
            if class_name not in classes or difficult == "1":
                continue

            cls_id = classes.index(class_name)
            xmlbox = obj.find('bndbox')
            tlx, tly = int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text)
            brx, bry = int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text)
            x = (tlx + brx) / 2 / image_w
            y = (tly + bry) / 2 / image_h
            w = (brx - tlx) / image_w
            h = (bry - tly) / image_h

            loc = [7 * x, 7 * y]
            loc_i = int(loc[1])
            loc_j = int(loc[0])
            y = loc[1] - loc_i
            x = loc[0] - loc_j

            if label_matrix[loc_i, loc_j, 24] == 0:
                label_matrix[loc_i, loc_j, cls_id] = 1
                label_matrix[loc_i, loc_j, 20:24] = [x, y, w, h]
                label_matrix[loc_i, loc_j, 24] = 1  # response

        return image, label_matrix

    def data_generation(self, batch_datasets): #파일경로가 들어있음
        images = []
        labels = []

        for file_path in batch_datasets: # 각 경로의 파일 읽어서 images, labels에 넣어줌(배치 단위로)
            image, label = self.read(file_path)
            images.append(image)
            labels.append(label)
        """
        들어온 batch_datasets 만큼의 이미지와 라벨링 데이터를 전달할 수 있도록 하세요.
        예를 들어 전체 데이터 중 batch_size 가 10이라면 batch_datsets에는 전체 데이터 / 10 만큼의 이미지 파일 경로들이 담겨있습니다.
        """
        return np.array(images), np.array(labels)

In [ ]:
import tensorflow
from tensorflow.keras import datasets, layers, models, activations, losses, optimizers, metrics, utils
import model
import loss
import data_generator


if __name__ == "__main__":
    # train_images, train_labels = dataset.voc_load_data("./VOCdevkit/VOC2007/JPEGImages", "./VOCdevkit/VOC2007/Annotations", batch=1000)
    
    yolo = model.create_yolo()
    
    yolo.compile(optimizer=optimizers.Adam(), loss=loss.yolo_loss)
    train_generator = data_generator.SequenceData("train",
                                                  "./VOC2007/images",
                                                  "./VOC2007/labels",
                                                  (448, 448, 3),
                                                  batch_size=10,
                                                  shuffle=True)

    yolo.fit_generator(generator=train_generator,
                       steps_per_epoch=len(train_generator),
                       epochs=1,
                       use_multiprocessing=True,
                       workers=4)
                       
    x, y = train_generator[0]
    print('첫 번째 데이터 셋:', x.shape, y.shape)

## Callback으로 특정 주기마다 모델 저장

* yolo.fit_generator에 callbacks 인자가 있다
* train의 시작과 끝, 매 epoch의 시작과 끝에 모델 저장 설정가능
* 특정 loss 도달했을 때 학습 멈추고 모델 저장도 가능  

main.py에 넣음

In [ ]:
import tensorflow
from tensorflow.keras import datasets, layers, models, activations, losses, optimizers, metrics, utils, callbacks
import model
import loss
import data_generator

# 매 epoch 마다 모델을 저장하는 callback
class MyCallback(callbacks.Callback):
    #on_train_begin(): 학습이 시작될 때 호출
    #on_train_end(): 학습이 끝날 때 호출
    #on_epoch_begin(): epoch가 시작될 때 호출
    #on_epoch_end(): epoch가 끝날 때 호출
    #위의 네 개의 함수를 오버라이딩 가능
    def on_train_begin(self, logs=None):
        print("학습이 시작되었습니다.")
        if logs is not None:
            print(logs)

    def on_train_end(self, logs=None):
        print("학습이 완료되었습니다.")
        if logs is not None:
            print(logs)

    def on_epoch_begin(self, epoch, logs=None): #f-string formatting
        print(f"{epoch}회차 학습이 시작되었습니다.")
        if logs is not None:
            print(logs)

    def on_epoch_end(self, epoch, logs=None):
        print(f"{epoch}회차 학습이 완료되었습니다.")
        self.model.save_weights(f"my-yolov1-{epoch}.hdf5")
        print(f"my-yolov1-{epoch}.hdf5 저장 완료")
        if logs is not None:
            print(logs)
        
    pass


if __name__ == "__main__":
    yolo = model.create_yolo()
    
    yolo.compile(optimizer=optimizers.Adam(), loss=loss.yolo_loss)
    train_generator = data_generator.SequenceData("train",
                                                  "./VOC2007/images",
                                                  "./VOC2007/labels",
                                                  (448, 448, 3),
                                                  batch_size=10,
                                                  shuffle=True)

    yolo.fit_generator(
        generator=train_generator,
        steps_per_epoch=len(train_generator),
        epochs=3,
        use_multiprocessing=True,
        workers=4,
        callbacks=[MyCallback()]
    )

## 출력 텐서 decoding

* yolo의 출력텐서는 7x7x30으로 이루어짐

![](https://elice-api-cdn.azureedge.net/api-attachment/attachment/72bcbfe646e145baa4fddc2236a8b2ac/image.png)  

* 이처럼 7x7 그리드를 순회하며 output tensor로 바운딩박스를 그린다
   총 7x7x2, 98개의 바운딩 박스가 그려졌을 것(NMS를 안써서 개수 안줄어듦)

decoder.py

In [ ]:
import numpy


# 출력 텐서를 해석하여 x, y, w, h로 표현되는 바운딩 박스를 리턴하는 함수

def decode(y, image_w, image_h):# y는 (1,7,7,30) shape - 1은 배치 사이즈
    boxes = []
    
    
    # 7 x 7의 그리드 셀을 순회
    for i in range(7):
        for j in range(7):
            grid_vector = y[0]
                
            # 두 개의 Anchorbox
            # AnchorBox는 = [x, y, w, h] 로 구성
            # 이때 x, y는 셀 안에서의 중심 좌표!
            anchor_boxA = grid_vector[i, j, 20:25]
            anchor_boxB = grid_vector[i, j, 25:]
            box1 = anchor_boxA.copy()
            box2 = anchor_boxB.copy()
                
            # 첫번째 Anchor Box의 셀 안에서의 좌표를 이용하여
            # 원래 이미지에서 중심 좌표로 변환 합니다.
            # 박스의 가로, 세로 역시 마찬가지로 좌표를 원본 이미지에 맞게 조정
            box1[0] = (j + box1[0]) / 7 * image_w #이미지 상의 x좌표
            box1[1] = (j + box1[1]) / 7 * image_h
            box1[2] = box1[2] * image_w
            box1[3] = box1[3] * image_h

            # 두번째 Anchor Box의 셀 안에서의 좌표를 이용하여
            # 원래 이미지에서 중심 좌표로 변환
            # 박스의 가로, 세로 역시 마찬가지로 좌표를 원본 이미지에 맞게 조정
            box2[0] = (j + box2[0]) / 7 * image_w
            box2[1] = (j + box2[1]) / 7 * image_h
            box2[2] = box2[2] * image_w
            box2[3] = box2[3] * image_h

            # 첫번째 박스의 중심좌표를 tlx, tly로 변환
            box1[0] -= (box1[2] / 2)
            box1[1] -= (box1[3] / 2)

            # 두번째 박스의 중심좌표를 tlx, tly으로 변환
            box2[0] -= (box2[2] / 2)
            box2[1] -= (box2[3] / 2)

            boxes.append(box1)
            boxes.append(box2)

    return boxes

In [ ]:
import tensorflow
from tensorflow.keras import datasets, layers, models, activations, losses, optimizers, metrics, utils
import model
import loss
import decoder
import cv2
import numpy


from elice_utils import EliceUtils


elice_utils = EliceUtils()


if __name__ == "__main__":
    yolo = model.create_yolo()
    yolo.summary()

    yolo.load_weights("weights_checkpoint_1.hdf5")

    input_shape = (1, 448, 448, 3)

    image = cv2.imread("./000001.jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, input_shape[1:3])
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_RGB2BGR)
    image_h, image_w = image.shape[0:2]

    # 모델 입력에 맞게 reshaping 후 정규화
    image = numpy.reshape(image, input_shape)
    image = image / 255.

    y = yolo.predict(image, batch_size=1)
    boxes = decoder.decode(y, image_h, image_w)


    for box in boxes:
        cv2.rectangle(draw, (int(box[0]), int(box[1])),
                      (int(box[0] + box[2]), int(box[1] + box[3])), (255, 0, 255))


    cv2.imwrite("result.jpg", draw)    
    elice_utils.send_image('result.jpg')

## 출력 텐서 디코딩에 NMS 적용

![](https://elice-api-cdn.azureedge.net/api-attachment/attachment/cd88ad0b4ac1438cb89df943f97ab753/image.png)
* 이렇게 박스들이 confidence threshold보다 낮을 경우 삭제되고 NMS 적용으로 기준 박스와의 IOU가 IoU Threshold보다 높으면 삭제되면서 걸려진다  
decoder.py

In [ ]:
import numpy


classes_num = {'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4, 'bus': 5,
               'car': 6, 'cat': 7, 'chair': 8, 'cow': 9, 'diningtable': 10, 'dog': 11,
               'horse': 12, 'motorbike': 13, 'person': 14, 'pottedplant': 15, 'sheep': 16,
               'sofa': 17, 'train': 18, 'tvmonitor': 19}

classes = list(classes_num.keys())



def intersection_over_union(box1, box2):

    # 교집합 부분의 top left 좌표와 bottom right 좌표 계산
    x1 = numpy.maximum(box1[0], box2[0])
    y1 = numpy.maximum(box1[1], box2[1])
    x2 = numpy.minimum(box1[0] + box1[2], box2[0] + box2[2])
    y2 = numpy.minimum(box1[1] + box1[3], box2[1] + box2[3])

    # 교집합의 넒이
    intersection = numpy.maximum(x2 - x1, 0) * numpy.maximum(y2 - y1, 0)

    # 박스1의 넓이와 박스2의 넓이
    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]

    # 두 박스의 넒이를 더한뒤 교집합 영역 넓이를 뺴면 합영역
    union = box1_area + box2_area - intersection

    # iou 계산
    iou = intersection / union
    return iou


def decode(y, image_shape, class_confidence_threshold=0.2, iou_threshold=0.2):
    boxes, names = [], [] #박스에 클래스 이름 써있도록
    grid_vector = y[0]
    
    # NMS 진행 : IOU가 높아 겹치는 박스를 제거
    # 아래 NMS 배열은 98개의 클래스 별 신뢰도를 담을 행렬
    # 0 ~ 20 행까지는 클래스 별 신뢰도를, 20 ~ 25행 까지는 AnchorBox의 좌표와 신뢰도 값을 저장
 
    nms = numpy.zeros((25, 98))
    for i in range(7):
        for j in range(7):
            box_num = i * 7 + j
            # AnchorBox A
            nms[:20, box_num] = grid_vector[i, j, 0:20]
            # 박스의 confidence를 클래스 confidence에 곱하여 
            # 박스의 위치와 클래스 모두 고려될 수 있도록
            nms[:20, box_num] *= grid_vector[i, j, 24]
            nms[20:, box_num] = grid_vector[i, j, 20:25]
                                
            # 박스 좌표 변환 : 그리드 셀에서 좌표를 -> x, y, w, h로 image_shape에 맞게
            nms[20, box_num] = (j + grid_vector[i, j, 20]) / 7 * image_shape[1]
            nms[21, box_num] = (i + grid_vector[i, j, 21]) / 7 * image_shape[0]
            nms[22, box_num] = grid_vector[i, j, 22] * image_shape[1]
            nms[23, box_num] = grid_vector[i, j, 23] * image_shape[0]
                  
            # 박스의 중심좌표를 x1, y1으로 변환
            nms[20, box_num] -= (nms[22, box_num] / 2)
            nms[21, box_num] -= (nms[23, box_num] / 2)
            

            # AnchorBox B
            nms[:20, box_num + 1] = grid_vector[i, j, 0:20]
            # 박스의 confidence를 클래스 confidence에 곱하여 
            # 박스의 위치와 클래스 모두 고려될 수 있도록
            nms[:20, box_num + 1] *= grid_vector[i, j, -1]
            nms[20:, box_num + 1] = grid_vector[i, j, 25:]
            
            # 박스 좌표 변환 : 그리드 셀에서 좌표를 -> x, y, w, h로 image_shape에 맞게
            nms[20, box_num + 1] = (j + grid_vector[i, j, 25]) / 7 * image_shape[1]
            nms[21, box_num + 1] = (i + grid_vector[i, j, 26]) / 7 * image_shape[0]
            nms[22, box_num + 1] = grid_vector[i, j, 27] * image_shape[1]
            nms[23, box_num + 1] = grid_vector[i, j, 28] * image_shape[0]
                    
            # 박스의 중심좌표를 x1, y1으로 변환
            nms[20, box_num + 1] -= (nms[22, box_num + 1] / 2)
            nms[21, box_num + 1] -= (nms[23, box_num + 1] / 2)

    # 아래 주석을 해제하면, nms 배열에 클래스 별 신뢰도 값이 복사되었는지 알 수 있음
    # for c in range(20):
    #     for k in range(0, 98):
    #         print(f"{classes[c]}에 대한 {k} 번째 박스 신뢰도는 {nms[c, k]}")

    # 모든 클래스 마다
    for class_id in range(nms.shape[0] - 5):
        # 모든 박스별로
        for box_order in range(nms.shape[1]):

            # 클래스의 신뢰도가 낮으면 주어진 클래스 threshold 보다 낮으면
            # 해당 클래스 신뢰도를 0으로
            if nms[class_id, box_order] < class_confidence_threshold:            
                nms[class_id, box_order] = 0
                pass

        # class confidence값에 따라 소팅하여(내림차순)
        # 클래스 별로 해당 클래스에서 IOU가 높은 것을 제거하도록 
        # 해당 박스의 클래스에 신뢰도 0으로
        candidates = nms[class_id, :].argsort()[::-1]
        for i in range(candidates.shape[0]): #기준박스에 대한 loop
            for j in range(i + 1, candidates.shape[0]): #기준박스와의 비교
                box1 = nms[20:24, candidates[i]]
                box2 = nms[20:24, candidates[j]]
                iou = intersection_over_union(box1, box2) #기존에 구현한 것과 iou 인자 조금 다름
                
                if iou > iou_threshold:
                    nms[class_id, candidates[j]] = 0
                    
    # 아래 주석을 해제하면 IOU로 제거된 이후의 신뢰도 출력
    # for c in range(20):
    #     for k in range(0, 98):
    #         print(f"{classes[c]}에 대한 {k} 번째 박스 신뢰도는 {nms[c, k]}")
    
    # 이제 남은 박스들 중 점수가 0이상인 박스들만
    # 좌표를 x. y, w, h로 변환
    for box_num in range(nms.shape[1]):
        class_id = numpy.argmax(nms[:20, box_num])
        confidence = nms[class_id, box_num]
                
        if confidence > 0:
            box = nms[20:24, box_num].copy()
            boxes.append(box)
            names.append(classes[class_id])
    
    return boxes, names

## 실신 감지 알고리즘(w/ yolov3tiny)
* 침대가 있는 영역을 rectangle로 잡아 roi로 설정
* 사람 객체의 바운딩박스 중심이 roi에 포함되면 바운딩박스 색 변함
* 그러고 5초 이상(150프레임) 지속되면 falldown으로 감지
* 다시 roi 밖으로 나가면 실신 아니었거나 풀린 것

In [ ]:
import cv2
import tensorflow as tf
from yolov3.models import YoloV3Tiny

from yolov3.dataset import transform_images
from yolov3.utils import draw_outputs


from answer import answer


def convert_outputs(outputs):
    ret_boxes, ret_scores, ret_classes = [], [], []

    boxes, scores, classes, nums = outputs
    boxes, scores, classes, nums = list(boxes[0]), list(scores[0]), list(classes[0]), nums[0]

    for i in range(nums):
        box, score, class_idx = boxes[i], scores[i], classes[i]
        ret_boxes.append(box)
        ret_scores.append(score)
        ret_classes.append(class_names[class_idx])

    return ret_boxes, ret_scores, ret_classes

# 박스의 중간 좌표가 roi안에 들어있으면 1, 아니면 0
def isInRoi(cx, cy, roi): 
    return (roi[0]<cx and cx<roi[2] and roi[1]<cy and cy < roi[3])

if __name__ == '__main__':
    class_names = [c.strip() for c in open('./coco.names').readlines()]
    yolo = YoloV3Tiny(classes=len(class_names))
    yolo.load_weights('./checkpoints/yolov3-tiny.tf')

    vid = cv2.VideoCapture("cam5.avi")

    falldown_start, falldown_end = 0, 0
    frame_number = 0

    roi = [250,220,400,320]
    hit, falldown_start, falldown_end = 0, 0, 0
    status = "normal"

    while vid.isOpened():
        _, img = vid.read()

        if img is None:
            continue

        img_in = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_in = tf.expand_dims(img_in, 0)
        img_in = transform_images(img_in, 416)
        boxes, scores, classes = convert_outputs(yolo.predict(img_in))
        color = (255, 0, 0)
        
        
        #boxes에는 박스의 (x1,y2), (x2,y2) 좌표가 실수로 있음
        #scores에는 박스의 confidence 값이 있고, classes에는 str타입에 박스의 클래스 이름
     
      

        for box in boxes:
            # box의 중심 좌표
            cx = (box[0] * img.shape[1] + box[2] * img.shape[1]) // 2
            cy = (box[1] * img.shape[0] + box[3] * img.shape[0]) // 2

            cv2.circle(img, (int(cx), int(cy)), 10, (255, 255, 128), 1)

            in_roi = isInRoi(cx, cy, roi)
            if status == "mornal" and in_roi:
                color = (0, 0, 128)
                hit += 1
                status = "test"
            elif status == "test" and in_roi:
                hit += 1
                if hit > 150: #30fps영상에서의 5초 이상!
                    status = "falldown"
                    color = (0, 0, 255)
                    falldown_start = frame_number
                    print(f"실신시작 - {falldown_start}")
            elif status == "falldown":
                if in_roi:
                    color = (0, 0, 255)
                else:
                    color = (255, 0, 0)
                    status = "normal"
                    hit = 0
                    falldown_end = frame_number
                    print(f"실신시작 - {falldown_end}")

        
        cv2.rectangle(img, (roi[0], roi[1]), (roi[2], roi[3]), color, 2)
        
        cv2.putText(img, f"{frame_number}",(20, 20),
                    cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, color, 2)
       
                    
        for box, score, class_name in zip(boxes, scores, classes):
            box[0] *= img.shape[1]
            box[1] *= img.shape[0]
            box[2] *= img.shape[1]
            box[3] *= img.shape[0]
            cv2.rectangle(img, (int(box[0]), int(box[1])),
                               (int(box[2]), int(box[3])), color, 2)
            cv2.putText(img, f"{class_name} {score:.3f}",
                        (int(box[0]), int(box[1])),
                        cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, color, 2)

        frame_number += 1
        cv2.imshow('output', img)
        if cv2.waitKey(1) == ord('q'):
            break

    cv2.destroyAllWindows()
    
    # 정답을 출력합니다.
    answer(falldown_start, falldown_end)
    